In [41]:
import pandas as pd
import sqlite3

In [42]:
# Load data from CSV files
providers = pd.read_csv("providers_data.csv")
receivers = pd.read_csv("receivers_data.csv")
food_listings = pd.read_csv("food_listings_data.csv")
claims = pd.read_csv("claims_data.csv")

In [9]:
providers

,Provider_ID,Name,Type,Address,City,Contact
0,1,Gonzales-Cochran,Supermarket,"74347 Christopher Extensions\nAndreamouth, OK ...",New Jessica,+1-600-220-0480
1,2,"Nielsen, Johnson and Fuller",Grocery Store,"91228 Hanson Stream\nWelchtown, OR 27136",East Sheena,+1-925-283-8901x6297
2,3,Miller-Black,Supermarket,"561 Martinez Point Suite 507\nGuzmanchester, W...",Lake Jesusview,001-517-295-2206
3,4,"Clark, Prince and Williams",Grocery Store,"467 Bell Trail Suite 409\nPort Jesus, IA 61188",Mendezmouth,556.944.8935x401
4,5,Coleman-Farley,Grocery Store,"078 Matthew Creek Apt. 319\nSaraborough, MA 53978",Valentineside,193.714.6577
...,...,...,...,...,...,...
995,996,"Vasquez, Ruiz and Flowers",Restaurant,"84308 Justin Stravenue\nNew Amberside, NE 53447",Williamview,+1-319-378-7627x0682
996,997,Garza-Williams,Catering Service,"08864 Figueroa Radial Suite 948\nJennaberg, AZ...",East Rossside,001-924-441-3963x746
997,998,Novak Group,Grocery Store,"934 Zachary Run\nMelissamouth, WY 02729",Joshuastad,(903)642-1969x3300
998,999,Moody Ltd,Grocery Store,"17580 Ernest Hills\nLake Michaelmouth, OR 56416",Stevenchester,637.300.3664x4880


In [43]:
import mysql.connector as p
print("Connector works!")


Connector works!


In [54]:

import mysql.connector

conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="12345",

)

cursor = conn.cursor()
cursor.execute("SHOW DATABASES;")

for db in cursor:
    print(db)


('food_data',)
('information_schema',)
('mysql',)
('performance_schema',)
('sys',)
('test',)


In [55]:
cursor.execute("CREATE DATABASE IF NOT EXISTS food_data")
print("MySQL database 'food_data' created successfully!")


MySQL database 'food_data' created successfully!


In [56]:
cursor.execute("use food_data")

In [47]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS providers (
        Provider_ID INT PRIMARY key,
        Name VARCHAR(255),
        Type VARCHAR(100),
        Address TEXT,
        City VARCHAR(100),
        Contact VARCHAR(50)
    )
""")


In [57]:
cursor.execute("""
 CREATE TABLE IF NOT EXISTS  Receivers (
     Receiver_ID INT Primary Key,
     Name Varchar(255),
     Type Varchar(266),
     City Varchar(200),
     Contact varchar(50) 
    )
""")

In [58]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS FoodListings (
    Food_ID INT PRIMARY KEY,
    Food_Name VARCHAR(255),
    Quantity INT,
    Expiry_Date DATE,
    Provider_ID INT,
    Provider_Type VARCHAR(255),
    Location VARCHAR(255),
    Food_Type VARCHAR(255),
    Meal_Type VARCHAR(255)
    )
""")

In [59]:
cursor.execute("""
    create table if not exists Claims(
    Claim_ID INT PRIMARY KEY,
    Food_ID INT,
    Receiver_ID INT,
    Status VARCHAR(255),
    Timestamp DATETIME)
    """
)


In [53]:
# Insert data using iterrows()
for index, row in providers.iterrows():
    cursor.execute("""INSERT INTO providers (Provider_ID, Name, Type, Address, City, Contact)
        VALUES (%s, %s, %s, %s, %s, %s)""", tuple(row))

IntegrityError: 1062 (23000): Duplicate entry '1' for key 'providers.PRIMARY'

In [41]:
for index, row in receivers.iterrows():
    cursor.execute("""
        INSERT INTO Receivers (Receiver_ID, Name, Type, City, Contact)
        VALUES (%s, %s, %s, %s, %s)""", tuple(row))

In [52]:
# Fix date format
food_listings["Expiry_Date"] = pd.to_datetime(food_listings["Expiry_Date"]).dt.strftime('%Y-%m-%d')

# Insert row by row
for index, row in food_listings.iterrows():
    cursor.execute("""
        INSERT INTO FoodListings(Food_ID, Food_Name, Quantity, Expiry_Date, Provider_ID, Provider_Type, Location, Food_Type, Meal_Type)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, tuple(row))

conn.commit()


In [55]:
# Convert Timestamp to correct MySQL format
claims["Timestamp"] = pd.to_datetime(claims["Timestamp"]).dt.strftime('%Y-%m-%d %H:%M:%S')

# Insert row by row
for index, row in claims.iterrows():
    cursor.execute("""
        INSERT INTO Claims (Claim_ID, Food_ID, Receiver_ID, Status, Timestamp)
        VALUES (%s, %s, %s, %s, %s)
    """, tuple(row))

conn.commit()


In [ ]:
#SQL QUeries

📌 Which city has the highest number of food providers?

In [60]:
# SQL query to get the city with the highest number of providers
query = """
    SELECT City, COUNT(*) AS Provider_Count
    FROM providers
    GROUP BY City
    ORDER BY Provider_Count DESC
    LIMIT 1;
"""

cursor.execute(query)
result = cursor.fetchall()

# Convert result into a DataFrame for better readability
df = pd.DataFrame(result, columns=["City", "Provider_Count"])
df



,City,Provider_Count
0,New Carol,3


In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def get_data(query, params=None):  
    conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="12345",

) 
    if params:
        df = pd.read_sql_query(query, conn, params=params)
    else:
        df = pd.read_sql_query(query, conn)
    conn.close()
    return df

# Streamlit App Title
st.set_page_config(page_title="waste food management system", layout="wide")

# Sidebar for navigation
st.sidebar.title("Navigation")
page = st.sidebar.radio("Go to", ["Project Introduction","CRUD operations" "SQL Queries", "Waste Food Data Visualization", "Creator Info"])

# -------------------------------- PAGE 1: Introduction --------------------------------
if page == "Project Introduction":
    st.title("LOcal Food Waste Management")
    st.subheader("A Streamlit App for Exploring Local waste Food Mangement Trends")
    st.write("""
    This project analyzes Food wastage is a significant issue, with many households 
    and restaurants discarding surplus food while numerous people struggle with food insecurity. This project aims to develop a Local Food Wastage Management System, where:
        Restaurants and individuals can list surplus food.
        NGOs or individuals in need can claim the food.
        SQL stores available food details and locations.

    **Database Used:** `Food_database.SQL`
    """)
# -------------------------------- PAGE 2: Crud operations --------------------------------
elif page == "CRUD Operations":
    def get_data(query, params=()):     
        conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="12345",
    )
        df = pd.read_sql_query(query, conn, params=params)
        conn.close()
        return df

# --- Execute Query ---
def execute_query(query, params=()):
    conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="12345",

)
    conn = get_connection()
    cur = conn.cursor()
    cur.execute(query, params)
    conn.commit()
    conn.close()

# --- CRUD Operations Page ---
def crud_operations():
    st.title("🔄 CRUD Operations")

    tables = ["providers", "receivers", "food_listings", "claims"]
    table = st.selectbox("Select Table", tables)

    df = get_data(f"SELECT * FROM {table}")
    st.write(f"### Current Data in {table}", df)

    st.subheader("➕ Create Record")
    with st.form(f"add_{table}"):
        new_data = {}
        for col in df.columns[1:]:  # skip ID (auto-increment usually)
            new_data[col] = st.text_input(f"Enter {col}")
        submitted = st.form_submit_button("Add Record")
        if submitted:
            cols = ", ".join(new_data.keys())
            placeholders = ", ".join(["?"] * len(new_data))
            execute_query(
                f"INSERT INTO {table} ({cols}) VALUES ({placeholders})",
                tuple(new_data.values())
            )
            st.success("✅ Record added successfully!")

    st.subheader("✏️ Update Record")
    record_id = st.selectbox("Select ID to Update", df.iloc[:, 0])
    with st.form(f"update_{table}"):
        updated_data = {}
        for col in df.columns[1:]:
            updated_data[col] = st.text_input(f"New {col}", value=str(
                df.loc[df.iloc[:, 0] == record_id, col].values[0]
            ))
        update_btn = st.form_submit_button("Update Record")
        if update_btn:
            set_clause = ", ".join([f"{col} = ?" for col in updated_data.keys()])
            execute_query(
                f"UPDATE {table} SET {set_clause} WHERE {df.columns[0]} = ?",
                tuple(updated_data.values()) + (record_id,)
            )
            st.success("✅ Record updated successfully!")

    st.subheader("🗑️ Delete Record")
    delete_id = st.selectbox("Select ID to Delete", df.iloc[:, 0], key=f"delete_{table}")
    if st.button("Delete Record"):
        execute_query(f"DELETE FROM {table} WHERE {df.columns[0]} = ?", (delete_id,))
        st.success("✅ Record deleted successfully!")
crud_operations()
# -------------------------------- PAGE 3: SQL Queries --------------------------------

# Handle queries needing user input
if selected_query == "4. Contact Info of Providers in a City":
    city = st.text_input("Enter City")
    if city:
        params = (city,)

# Run query
if params:
    query_result = get_data(queries[selected_query], params=params)
else:
    query_result = get_data(queries[selected_query])

st.write("### Query Result:")
st.dataframe(query_result)

# -------------------------------- PAGE 4: SQL --------------------------------

elif page == "Waste Food Data Visualization":
    st.title("📊 Waste Food Data Visualization")
        def get_data(query, params=()):
    conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="12345",
)
    df = pd.read_sql_query(query, conn, params=params)
    conn.close()
    return df

# --- Execute Query ---

# --- CRUD Operations Page ---
queries = {
    "1. Food Providers per City": """
        SELECT Location AS City, COUNT(*) AS Num_Providers
        FROM providers
        GROUP BY Location
        ORDER BY Num_Providers DESC;
    """,

    "2. Receivers per City": """
        SELECT Location AS City, COUNT(*) AS Num_Receivers
        FROM receivers
        GROUP BY Location
        ORDER BY Num_Receivers DESC;
    """,

    "3. Top Provider Type by Contributions": """
        SELECT Provider_Type, SUM(f.Quantity) AS Total_Quantity
        FROM providers p
        JOIN food f ON p.Provider_ID = f.Provider_ID
        GROUP BY Provider_Type
        ORDER BY Total_Quantity DESC;
    """,

    "4. Contact Info of Providers in a City": """
        SELECT Name, Phone, Email, Location
        FROM providers
        WHERE Location = %s;   -- pass city dynamically
    """,

    "5. Receivers with Most Food Claimed": """
        SELECT r.Name, SUM(c.Quantity) AS Total_Claimed
        FROM receivers r
        JOIN claims c ON r.Receiver_ID = c.Receiver_ID
        GROUP BY r.Name
        ORDER BY Total_Claimed DESC
        LIMIT 10;
    """,

    "6. Total Quantity of Food Available": """
        SELECT SUM(Quantity) AS Total_Available
        FROM food
        WHERE Status = 'Available';
    """,

    "7. City with Highest Number of Food Listings": """
        SELECT f.Location, COUNT(*) AS Num_Listings
        FROM food f
        GROUP BY f.Location
        ORDER BY Num_Listings DESC;
    """,

    "8. Most Commonly Available Food Types": """
        SELECT Food_Type, COUNT(*) AS Frequency
        FROM food
        GROUP BY Food_Type
        ORDER BY Frequency DESC;
    """,

    "9. Number of Claims per Food Item": """
        SELECT f.Food_Name, COUNT(c.Claim_ID) AS Num_Claims
        FROM food f
        LEFT JOIN claims c ON f.Food_ID = c.Food_ID
        GROUP BY f.Food_Name
        ORDER BY Num_Claims DESC;
    """,

    "10. Provider with Highest Successful Claims": """
        SELECT p.Name, COUNT(c.Claim_ID) AS Successful_Claims
        FROM providers p
        JOIN food f ON p.Provider_ID = f.Provider_ID
        JOIN claims c ON f.Food_ID = c.Food_ID
        WHERE c.Status = 'Approved'
        GROUP BY p.Name
        ORDER BY Successful_Claims DESC
        LIMIT 10;
    """,

    "11. Claim Status Distribution": """
        SELECT Status,
               ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM claims), 2) AS Percentage
        FROM claims
        GROUP BY Status;
    """,

    "12. Average Quantity of Food Claimed per Receiver": """
        SELECT r.Name, ROUND(AVG(c.Quantity), 2) AS Avg_Claimed
        FROM receivers r
        JOIN claims c ON r.Receiver_ID = c.Receiver_ID
        GROUP BY r.Name
        ORDER BY Avg_Claimed DESC
        LIMIT 10;
    """,

    "13. Most Claimed Meal Type": """
        SELECT f.Meal_Type, COUNT(c.Claim_ID) AS Num_Claims
        FROM food f
        JOIN claims c ON f.Food_ID = c.Food_ID
        GROUP BY f.Meal_Type
        ORDER BY Num_Claims DESC;
    """,

    "14. Total Quantity Donated by Each Provider": """
        SELECT p.Name, SUM(f.Quantity) AS Total_Donated
        FROM providers p
        JOIN food f ON p.Provider_ID = f.Provider_ID
        GROUP BY p.Name
        ORDER BY Total_Donated DESC
        LIMIT 10;
    """,

    "15. Highest Demand City by Claims": """
        SELECT r.Location AS City, COUNT(c.Claim_ID) AS Total_Claims
        FROM receivers r
        JOIN claims c ON r.Receiver_ID = c.Receiver_ID
        GROUP BY r.Location
        ORDER BY Total_Claims DESC
        LIMIT 10;
    """
}

import matplotlib.pyplot as plt
import seaborn as sns

def visualize_query(selected_query, query_result):
    if query_result.empty:
        st.warning("⚠️ No data available for this query.")
        return

    # 1. Food Providers per City
    if selected_query == "1. Food Providers per City":
        st.write("### 🏢 Food Providers per City")
        plt.figure(figsize=(8,5))
        sns.barplot(data=query_result, x="City", y="Num_Providers")
        plt.xticks(rotation=45)
        st.pyplot(plt)

    # 2. Receivers per City
    elif selected_query == "2. Receivers per City":
        st.write("### 👥 Receivers per City")
        plt.figure(figsize=(8,5))
        sns.barplot(data=query_result, x="City", y="Num_Receivers")
        plt.xticks(rotation=45)
        st.pyplot(plt)

    # 3. Top Provider Type by Contributions
    elif selected_query == "3. Top Provider Type by Contributions":
        st.write("### 🍴 Provider Type Contributions")
        plt.figure(figsize=(6,4))
        sns.barplot(data=query_result, x="Provider_Type", y="Total_Quantity")
        st.pyplot(plt)

    # 4. Contact Info (table only, no chart)
    elif selected_query == "4. Contact Info of Providers in a City":
        st.write("📞 Provider Contact Info (Table Only)")

    # 5. Receivers with Most Food Claimed
    elif selected_query == "5. Receivers with Most Food Claimed":
        st.write("### 🏆 Top Receivers by Food Claimed")
        plt.figure(figsize=(10,6))
        sns.barplot(data=query_result.head(10), x="Total_Claimed", y="Name")
        st.pyplot(plt)

    # 6. Total Quantity of Food Available
    elif selected_query == "6. Total Quantity of Food Available":
        st.write("### 📦 Total Food Available")
        st.metric("Total Available Quantity", int(query_result["Total_Available"].iloc[0]))

    # 7. City with Highest Number of Food Listings
    elif selected_query == "7. City with Highest Number of Food Listings":
        st.write("### 🏙️ Top City by Food Listings")
        plt.figure(figsize=(6,4))
        sns.barplot(data=query_result, x="Location", y="Num_Listings")
        st.pyplot(plt)

    # 8. Most Commonly Available Food Types
    elif selected_query == "8. Most Commonly Available Food Types":
        st.write("### 🥗 Food Type Breakdown")
        plt.figure(figsize=(6,6))
        plt.pie(query_result["Frequency"], labels=query_result["Food_Type"], autopct="%1.1f%%")
        st.pyplot(plt)

    # 9. Number of Claims per Food Item
    elif selected_query == "9. Number of Claims per Food Item":
        st.write("### 📊 Claims per Food Item")
        plt.figure(figsize=(10,6))
        sns.barplot(data=query_result.head(15), x="Num_Claims", y="Food_Name")
        st.pyplot(plt)

    # 10. Provider with Highest Successful Claims
    elif selected_query == "10. Provider with Highest Successful Claims":
        st.write("### 🥘 Top Provider by Successful Claims")
        plt.figure(figsize=(8,5))
        sns.barplot(data=query_result, x="Successful_Claims", y="Name")
        st.pyplot(plt)

    # 11. Claim Status Distribution
    elif selected_query == "11. Claim Status Distribution":
        st.write("### ✅ Claim Status Distribution")
        plt.figure(figsize=(6,6))
        plt.pie(query_result["Percentage"], labels=query_result["Status"], autopct="%1.1f%%")
        st.pyplot(plt)

    # 12. Average Quantity Claimed per Receiver
    elif selected_query == "12. Average Quantity of Food Claimed per Receiver":
        st.write("### 📈 Average Claimed Quantity per Receiver")
        plt.figure(figsize=(10,6))
        sns.barplot(data=query_result.head(15), x="Avg_Claimed", y="Name")
        st.pyplot(plt)

    # 13. Most Claimed Meal Type
    elif selected_query == "13. Most Claimed Meal Type":
        st.write("### 🍽️ Most Claimed Meal Type")
        plt.figure(figsize=(6,4))
        sns.barplot(data=query_result, x="Meal_Type", y="Num_Claims")
        st.pyplot(plt)

    # 14. Total Quantity Donated by Each Provider
    elif selected_query == "14. Total Quantity Donated by Each Provider":
        st.write("### 🥘 Provider Contributions")
        plt.figure(figsize=(10,6))
        sns.barplot(data=query_result.head(15), x="Total_Donated", y="Name")
        st.pyplot(plt)

    # 15. Highest Demand City by Claims
    elif selected_query == "15. Highest Demand City by Claims":
        st.write("### 🏙️ Highest Demand City (Claims)")
        plt.figure(figsize=(8,5))
        sns.barplot(data=query_result, x="City", y="Total_Claims")
        st.pyplot(plt)
selected_query = st.selectbox("Choose a Query", list(queries.keys()))
query_text = queries[selected_query]
query_result = get_data(query_text)

st.write("### Query Result (Table)")
st.dataframe(query_result)

# Call visualization function
visualize_query(selected_query, query_result)
# -------------------------------- PAGE 4: Creator Info --------------------------------
elif page == "Creator Info":
    st.title("👩‍💻 Creator of this Project")
    st.write("""
    **Developed by:** Saikiran Devunuri  
    **Skills:** Python, SQL, Data Analysis,Streamlit, Pandas    
    """)
    st.image("https://via.placeholder.com/150", caption="Your Profile Picture", width=150)

Overwriting app.py


In [79]:
%%writefile app2.py
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mysql.connector

# -----------------------------
# MySQL connection helpers
# -----------------------------
def get_connection():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password="12345",
        database="food_data"   
    )
def queries_dict():
    return {
        # --- Providers & Receivers ---
        "1. Food Providers per City": """
            SELECT City, COUNT(*) AS Num_Providers
            FROM providers
            GROUP BY City
            ORDER BY Num_Providers DESC;
            Limit 5;
        """,
        "2. Receivers per City": """
            SELECT City, COUNT(*) AS Num_Receivers
            FROM receivers
            GROUP BY City
            ORDER BY Num_Receivers DESC;
        """,
        "3. Top Provider Type by Contributions": """
            SELECT fl.Provider_Type, SUM(fl.Quantity) AS Total_Quantity
            FROM foodlistings fl
            GROUP BY fl.Provider_Type
            ORDER BY Total_Quantity DESC;
        """,
        "4. Contact Info of Providers in a City (input)": """
            SELECT Name, Contact, Address, City
            FROM providers
            WHERE City = %s;
        """,
        "5. Receivers with Most Food Claimed": """
            SELECT r.Name, SUM(fl.Quantity) AS Total_Claimed
            FROM claims c
            JOIN receivers r ON c.Receiver_ID = r.Receiver_ID
            JOIN foodlistings fl ON c.Food_ID = fl.Food_ID
            WHERE c.Status = 'Completed'
            GROUP BY r.Name
            ORDER BY Total_Claimed DESC;
        """,

        # --- Food Listings & Availability ---
        "6. Total Quantity of Food Available": """
            SELECT SUM(Quantity) AS Total_Available
            FROM foodlistings;
        """,
        "7. City with Highest Number of Food Listings": """
            SELECT Location, COUNT(*) AS Num_Listings
            FROM foodlistings
            GROUP BY Location
            ORDER BY Num_Listings DESC;
        """,
        "8. Most Commonly Available Food Types": """
            SELECT Food_Type, COUNT(*) AS Frequency
            FROM foodlistings
            GROUP BY Food_Type
            ORDER BY Frequency DESC;
        """,

        # --- Claims & Distribution ---
        "9. Number of Claims per Food Item": """
            SELECT fl.Food_Name, COUNT(c.Claim_ID) AS Num_Claims
            FROM foodlistings fl
            LEFT JOIN claims c ON fl.Food_ID = c.Food_ID
            GROUP BY fl.Food_Name
            ORDER BY Num_Claims DESC;
        """,
        "10. Claims for a Specific Provider (input)": """
            SELECT p.Name, COUNT(c.Claim_ID) AS Successful_Claims
            FROM providers p
            JOIN foodlistings fl ON p.Provider_ID = fl.Provider_ID
            JOIN claims c ON fl.Food_ID = c.Food_ID
            WHERE c.Status = 'Completed' AND p.Name = %s
            GROUP BY p.Name;
        """,
        "11. Claim Status Distribution": """
            SELECT Status, COUNT(*) * 100.0 / (SELECT COUNT(*) FROM claims) AS Percentage
            FROM claims
            GROUP BY Status;
        """,

        # --- Analysis & Insights ---
        "12. Average Quantity of Food Claimed per Receiver": """
            SELECT r.Name, AVG(fl.Quantity) AS Avg_Claimed
            FROM claims c
            JOIN receivers r ON c.Receiver_ID = r.Receiver_ID
            JOIN foodlistings fl ON c.Food_ID = fl.Food_ID
            WHERE c.Status = 'Completed'
            GROUP BY r.Name
            ORDER BY Avg_Claimed DESC;
        """,
        "13. Most Claimed Meal Type": """
            SELECT fl.Meal_Type, COUNT(c.Claim_ID) AS Num_Claims
            FROM foodlistings fl
            JOIN claims c ON fl.Food_ID = c.Food_ID
            WHERE c.Status IN ('Completed','Pending','Cancelled')
            GROUP BY fl.Meal_Type
            ORDER BY Num_Claims DESC;
        """,
        "14. Total Quantity Donated by Each Provider": """
            SELECT p.Name, SUM(fl.Quantity) AS Total_Donated
            FROM providers p
            JOIN foodlistings fl ON p.Provider_ID = fl.Provider_ID
            GROUP BY p.Name
            ORDER BY Total_Donated DESC;
        """,
        "15. Highest Demand City by Claims": """
            SELECT r.City, COUNT(c.Claim_ID) AS Total_Claims
            FROM claims c
            JOIN receivers r ON c.Receiver_ID = r.Receiver_ID
            GROUP BY r.City
            ORDER BY Total_Claims DESC;
        """
    }


def visualize_query(selected_query, df):
    if df.empty:
        st.warning("⚠️ No data available for this query.")
        return

    if selected_query == "1. Food Providers per City":
        st.write("### 🏢 Providers per City")
        plt.figure(figsize=(9,5))
        sns.barplot(data=df, x="City", y="Num_Providers")
        plt.xticks(rotation=45)
        st.pyplot(plt)

    elif selected_query == "2. Receivers per City":
        st.write("### 👥 Receivers per City")
        plt.figure(figsize=(9,5))
        sns.barplot(data=df, x="City", y="Num_Receivers")
        plt.xticks(rotation=45)
        st.pyplot(plt)

    elif selected_query == "3. Top Provider Type by Contributions":
        st.write("### 🍴 Provider Type Contributions")
        plt.figure(figsize=(7,4))
        sns.barplot(data=df, x="Provider_Type", y="Total_Quantity")
        st.pyplot(plt)

    elif selected_query == "4. Contact Info of Providers in a City (input)":
        st.info("Showing provider contact info table for selected city.")

    elif selected_query == "5. Receivers with Most Food Claimed":
        st.write("### 🏆 Top Receivers by Food Claimed")
        plt.figure(figsize=(10,6))
        top = df.head(15)
        sns.barplot(data=top, x="Total_Claimed", y="Name")
        st.pyplot(plt)

    elif selected_query == "6. Total Quantity of Food Available":
        st.metric("Total Available Quantity", int(df["Total_Available"].iloc[0]))

    elif selected_query == "7. City with Highest Number of Food Listings":
        st.write("### 🏙️ Listings by City")
        plt.figure(figsize=(9,5))
        sns.barplot(data=df, x="Location", y="Num_Listings")
        plt.xticks(rotation=45)
        st.pyplot(plt)

    elif selected_query == "8. Most Commonly Available Food Types":
        st.write("### 🥗 Food Type Breakdown")
        plt.figure(figsize=(6,6))
        plt.pie(df["Frequency"], labels=df["Food_Type"], autopct="%1.1f%%")
        st.pyplot(plt)

    elif selected_query == "9. Number of Claims per Food Item":
        st.write("### 📦 Claims per Food Item")
        plt.figure(figsize=(10,6))
        top = df.head(20)
        sns.barplot(data=top, x="Num_Claims", y="Food_Name")
        st.pyplot(plt)

    elif selected_query == "10. Claims for a Specific Provider (input)":
        st.write("### ✅ Successful Claims for Provider")
        st.bar_chart(df.set_index(df.columns[0]))

    elif selected_query == "11. Claim Status Distribution":
        st.write("### 📊 Claim Status Distribution")
        plt.figure(figsize=(6,6))
        plt.pie(df["Percentage"], labels=df["Status"], autopct="%1.1f%%")
        st.pyplot(plt)

    elif selected_query == "12. Average Quantity of Food Claimed per Receiver":
        st.write("### 📈 Avg Claimed Quantity per Receiver")
        plt.figure(figsize=(10,6))
        top = df.head(20)
        sns.barplot(data=top, x="Avg_Claimed", y="Name")
        st.pyplot(plt)

    elif selected_query == "13. Most Claimed Meal Type":
        st.write("### 🍽️ Most Claimed Meal Type")
        plt.figure(figsize=(7,4))
        sns.barplot(data=df, x="Meal_Type", y="Num_Claims")
        st.pyplot(plt)

    elif selected_query == "14. Total Quantity Donated by Each Provider":
        st.write("### 🥘 Provider Contributions")
        plt.figure(figsize=(10,6))
        top = df.head(20)
        sns.barplot(data=top, x="Total_Donated", y="Name")
        st.pyplot(plt)

    elif selected_query == "15. Highest Demand City by Claims":
        st.write("### 🏙️ Highest Demand Cities (Claims)")
        plt.figure(figsize=(9,5))
        sns.barplot(data=df, x="City", y="Total_Claims")
        plt.xticks(rotation=45)
        st.pyplot(plt)
# -----------------------------
def get_data(query, params=()):
    conn = get_connection()
    df = pd.read_sql(query, conn, params=params)
    conn.close()
    return df

def execute_query(query, params=()):
    conn = get_connection()
    cur = conn.cursor()
    cur.execute(query, params)
    conn.commit()
    cur.close()
    conn.close()

# -----------------------------
# App config
# -----------------------------
st.set_page_config(page_title="Waste Food Management System", layout="wide")

# -----------------------------
# Sidebar
# -----------------------------
st.sidebar.title("Navigation")
page = st.sidebar.radio(
    "Go to",
    ["Project Introduction", "CRUD Operations", "SQL Queries", "Waste Food Data Visualization", "Creator Info"]
)

# -----------------------------
# Page 1: Introduction
# -----------------------------
if page == "Project Introduction":
    st.title("Local Food Waste Management")
    st.subheader("A Streamlit App for Exploring Local Waste Food Management Trends")
    st.write(
        """
This project analyzes food wastage and distribution. Restaurants and individuals can list surplus food; NGOs or individuals can claim it.
Data is stored in MySQL tables: `providers`, `receivers`, `food_listings`, `claims`.
        """
    )
    st.markdown("**Database Used:** `food_database` (update in code if different)")

# -----------------------------
# Page 2: CRUD Operations
# -----------------------------
elif page == "CRUD Operations":

    st.title("🔄 CRUD Operations")

    tables = ["providers", "receivers", "food_listings", "claims"]
    table = st.selectbox("Select Table", tables)

    # Map primary key column names
    id_map = {
        "providers": "Provider_ID",
        "receivers": "Receiver_ID",
        "food_listings": "Food_ID",
        "claims": "Claim_ID",
    }
    id_col = id_map[table]

    # Load table
    df = get_data(f"SELECT * FROM {table}")
   

    # ----- CREATE -----
    st.subheader("➕ Create Record")
    if df.shape[1] > 0:
        with st.form(f"add_{table}"):
            new_vals = {}
            for col in df.columns:
                if col == id_col:
                    # Assume auto-increment; skip manual entry
                    continue
                new_vals[col] = st.text_input(f"Enter {col}")
            submitted = st.form_submit_button("Add Record")
            if submitted:
                if len(new_vals) == 0:
                    st.error("No columns to insert.")
                else:
                    cols = ", ".join(new_vals.keys())
                    placeholders = ", ".join(["%s"] * len(new_vals))
                    execute_query(
                        f"INSERT INTO {table} ({cols}) VALUES ({placeholders})",
                        tuple(new_vals.values())
                    )
                    st.success("✅ Record added successfully!")

    # ----- UPDATE -----
    if not df.empty:
        st.subheader("✏️ Update Record")
        record_id = st.selectbox("Select ID to Update", df[id_col].tolist())
        with st.form(f"update_{table}"):
            upd_vals = {}
            for col in df.columns:
                if col == id_col:
                    continue
                current_value = str(df.loc[df[id_col] == record_id, col].values[0])
                upd_vals[col] = st.text_input(f"New {col}", value=current_value)
            update_btn = st.form_submit_button("Update Record")
            if update_btn:
                set_clause = ", ".join([f"{col} = %s" for col in upd_vals.keys()])
                params = tuple(upd_vals.values()) + (record_id,)
                execute_query(
                    f"UPDATE {table} SET {set_clause} WHERE {id_col} = %s",
                    params
                )
                st.success("✅ Record updated successfully!")

        # ----- DELETE -----
        st.subheader("🗑️ Delete Record")
        delete_id = st.selectbox("Select ID to Delete", df[id_col].tolist(), key=f"delete_{table}")
        if st.button("Delete Record"):
            execute_query(f"DELETE FROM {table} WHERE {id_col} = %s", (delete_id,))
            st.success("✅ Record deleted successfully!")
    else:
        st.info("No records found in this table.")

# -----------------------------
# Shared: queries + visualizer
# -----------------------------

# Page 3: SQL Queries (table + viz)
elif page == "SQL Queries":
    st.title("SQL Query Results")
    queries = queries_dict()
    selected_query = st.selectbox("Choose a Query", list(queries.keys()))
    params = ()

    # Handle inputs
    if selected_query == "4. Contact Info of Providers in a City (input)":
        cities = get_data("SELECT DISTINCT City FROM providers")["City"].tolist()
        city = st.selectbox("Select City", cities)
        if city:
            params = (city,)

    if selected_query == "10. Claims for a Specific Provider (input)":
        providers = get_data("SELECT DISTINCT Name FROM providers")["Name"].tolist()
        provider = st.selectbox("Select Provider", providers)
        if provider:
            params = (provider,)

    # Run
    df = get_data(queries[selected_query], params)
    st.write("### Query Result")
    st.dataframe(df)


elif page=="Waste Food Data Visualization":
    st.title("📊 Waste Food Data Visualization")
    queries = queries_dict()
    selected_query = st.selectbox("Choose a Visualization Query", list(queries.keys()))
    params = ()

    if selected_query == "4. Contact Info of Providers in a City (input)":
        cities = get_data("SELECT DISTINCT City FROM providers")["City"].tolist()
        city = st.selectbox("Select City", cities)
        if city:
            params = (city,)

    if selected_query == "10. Claims for a Specific Provider (input)":
        providers = get_data("SELECT DISTINCT Name FROM providers")["Name"].tolist()
        provider = st.selectbox("Select Provider", providers)
        if provider:
            params = (provider,)

    df = get_data(queries[selected_query], params)
    st.write("### Visualization")
    visualize_query(selected_query, df)
    
# -----------------------------
# Page 5: Creator Info
# -----------------------------
elif page == "Creator Info":
    st.title("👩‍💻 Creator of this Project")
    st.write(
        """
    **Developed by:** Saikiran Devunuri  
    **Skills:** Python, SQL, Data Analysis, Streamlit, Pandas
        """
    )
    st.image("https://via.placeholder.com/150", caption="Profile Picture", width=150)


Overwriting app2.py


In [80]:
!streamlit run app2.py

^C
